 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

In [1]:
import pandas as pd
import synonyms
import jieba
import jieba.posseg as pseg

Model loaded succeed
>> Synonyms on loading ...
>> Synonyms vocabulary size: 125792


In [2]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [3]:
corpus_df = pd.read_csv('/Users/olap/Desktop/corpus.csv')
corpus_2_list = corpus_df[corpus_df['rate'] == 2]['token'].tolist()
corpus_3_list = corpus_df[corpus_df['rate'] == 3]['token'].tolist()

In [4]:
def title_token_rater(tokens):
    
    tokens_rate = list(map(lambda x:3 if x in corpus_3_list else(2 if x in corpus_2_list else 0), tokens))
        
    token_3_cnt = tokens_rate.count(3)
    token_2_cnt = tokens_rate.count(2)
    
    return [token_2_cnt, token_3_cnt]

In [5]:
def title_syn_rater(tokens):
    
    syn_3_cnt = 0
    syn_2_cnt = 0
    syn_cnt_list = []
    
    for token in tokens:
        syn = synonyms.nearby(token)[0]
        syn_rates = list(map(lambda x:3 if x in corpus_3_list else(2 if x in corpus_2_list else 0), syn))

        syn_cnt = [2 if syn_rates.count(3) < syn_rates.count(2) else(0 if syn_rates.count(3) == 0 and syn_rates.count(2) == 0 else 3)]
        syn_cnt_list += syn_cnt
        
    syn_3_cnt = syn_cnt_list.count(3)
    syn_2_cnt = syn_cnt_list.count(2)
     
    return [syn_2_cnt, syn_3_cnt]

In [6]:
def title_processor(title):
    
    tokens = list(set(list(jieba.cut_for_search(title))))
#     print(tokens)
    token_rate = title_token_rater(tokens)
    syn_rate = title_syn_rater(tokens)
    
    return token_rate + syn_rate

In [8]:
training_title_list = open('/Users/olap/Desktop/training_title_list.txt', 'r').read().split('\n')[:100]
title_feature_list = list(map(title_processor, training_title_list))

ture_label = open('/Users/olap/Desktop/training_set.txt', 'r').read().split('\n')
title_rate = open('/Users/olap/Desktop/title_rate.csv', 'r').read().split('\n')
true_label_list = list(map(lambda x:x.split(' ')[-1], ture_label))
true_label_list = [1 if label=='0' else label for label in true_label_list]
true_label_list = [0 if label=='100' else label for label in true_label_list][:-1]

label_list = list(map(lambda x:x.split(',')[-1], title_rate))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/qt/bjwlshwn1gv_4s_c_97p0s440000gn/T/jieba.cache
Loading model cost 0.940 seconds.
Prefix dict has been built succesfully.


In [11]:
title = pd.DataFrame(training_title_list)
feature = pd.DataFrame(title_feature_list)
true_label = pd.DataFrame(true_label_list)

noobs_df = pd.concat([title, feature, true_label], axis= 1)
noobs_df.columns = ['Title', 'token_2', 'token_3', 'syn_2', 'syn_3', 'true_label']

In [12]:
def processor_testing(title, rate):
    tokenized_title = list(set(list(jieba.cut_for_search(title))))
    for token in tokenized_title:
    #     print (token)
        token_syn = [token] + synonyms.nearby(token)[0]
        print (token_syn)
        
        if rate == 2:
            print (list(map(lambda x:x in corpus_2_list, token_syn)))
        if rate == 3:
            print (list(map(lambda x:x in corpus_3_list, token_syn)))

In [17]:
import numpy as np
X = np.array(noobs_df.iloc[:,1:5])
y = np.array(noobs_df.iloc[:,-1])

In [18]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
sss.get_n_splits(X, y)

1

In [19]:
for train_index, test_index in sss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [66 70 58 78 54 30  2 95 68  5 35 17 75 88 87 79 37  1 59 46 71 92 38 69 91
 15  7 62 10 76 45 72 53 64 31  8 56 80 11 63 89 77 22 57 16 42  4 65 40 36] TEST: [61 23 49 60 13 94 50  0 18 19 98 27 39 26 28 73 85 67 74 82 41 86 44 81 25
  3 93 32 96 97 52 84 33 99 83 55 14 24 21 43 51 12 29 34 47  6  9 20 90 48]


In [20]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [21]:
from sklearn import linear_model
model = linear_model.LogisticRegression()

In [22]:
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.90000000000000002

In [23]:
model.score(X_test, y_test)

0.90000000000000002

In [24]:
model_predict = pd.DataFrame(model.predict(X_test))
model_predict.columns = ['model_pred']
model_proba = pd.DataFrame(model.predict_proba(X_test))
model_proba.columns = ['proba_0', 'proba_1']

index = test_index.tolist()
noobs_df_test = noobs_df.iloc[index].loc[:, noobs_df.columns != 'label'].reset_index(drop = True)
noobs_df_test = pd.concat([noobs_df_test, model_predict, model_proba], axis= 1)

In [25]:
# keyword based model mistake cases
noobs_df_test[((noobs_df_test['token_3'] == 0) & (noobs_df_test['true_label'] == 1))
             | (noobs_df_test['token_3'] > 0) & (noobs_df_test['true_label'] == 0)]

,Title,token_2,token_3,syn_2,syn_3,true_label,model_pred,proba_0,proba_1
2,"舆情分析：兰陵一副校长致初中女生""怀孕"" - 案例库 - 华声舆情 - 华声在线",1,0,1,0,1,0,0.566844,0.433156
5,分手你还赚了一个处，你不亏！,1,0,1,0,1,0,0.566844,0.433156
21,网络惊现售卖原味丝袜二手内裤 交易背后不堪入目_新闻频道_中华网,3,0,2,0,1,1,0.109978,0.890022
22,外围女真实生活揭秘：一夜遍体鳞伤换来30万_食神的神_食神的神的和讯博客,0,0,1,0,1,0,0.805167,0.194833
29,大尺度无底线直播 – 【人人分享-人人网】,3,0,0,0,1,1,0.122308,0.877692
31,大麻会对生活造成危害吗？ - 知乎,0,0,0,0,1,0,0.814424,0.185576
34,【福利/欢乐向/AMV】新年就要啪啪啪 我可能遇见了假啪啪啪啪@FB_JAMes_MAD·A...,0,0,0,0,1,0,0.814424,0.185576


In [26]:
# ML model mistake cases
noobs_df_test[noobs_df_test['true_label'] != noobs_df_test['model_pred']]

,Title,token_2,token_3,syn_2,syn_3,true_label,model_pred,proba_0,proba_1
2,"舆情分析：兰陵一副校长致初中女生""怀孕"" - 案例库 - 华声舆情 - 华声在线",1,0,1,0,1,0,0.566844,0.433156
5,分手你还赚了一个处，你不亏！,1,0,1,0,1,0,0.566844,0.433156
22,外围女真实生活揭秘：一夜遍体鳞伤换来30万_食神的神_食神的神的和讯博客,0,0,1,0,1,0,0.805167,0.194833
31,大麻会对生活造成危害吗？ - 知乎,0,0,0,0,1,0,0.814424,0.185576
34,【福利/欢乐向/AMV】新年就要啪啪啪 我可能遇见了假啪啪啪啪@FB_JAMes_MAD·A...,0,0,0,0,1,0,0.814424,0.185576
